In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

In [59]:
df = pd.read_csv('/Users/Jackie/Desktop/py4e/Cohort Analysis/data.csv',encoding="ISO-8859-1",
                 )

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176427 entries, 0 to 176426
Data columns (total 14 columns):
Supporter ID               176427 non-null int64
Email                      176427 non-null object
Date record created        176427 non-null object
Date last activity         176427 non-null object
Campaign Number            176427 non-null int64
Campaign Type              176427 non-null object
Form name                  176427 non-null object
Date of activitiy          176427 non-null object
Donation amount            176427 non-null float64
Currency                   176427 non-null object
Credit/bank type           176427 non-null object
Donation frequency         176427 non-null object
USD currency conversion    176427 non-null float64
External Reference 8       121143 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 18.8+ MB


In [61]:
#drop one unnecessary column
df = df.drop(['Campaign Number','Campaign Type','External Reference 8'
              ,'Form name','Currency','Donation amount','Credit/bank type','Donation frequency'],axis='columns')

In [62]:
df.drop_duplicates(inplace = True)

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174845 entries, 0 to 176426
Data columns (total 6 columns):
Supporter ID               174845 non-null int64
Email                      174845 non-null object
Date record created        174845 non-null object
Date last activity         174845 non-null object
Date of activitiy          174845 non-null object
USD currency conversion    174845 non-null float64
dtypes: float64(1), int64(1), object(4)
memory usage: 9.3+ MB


In [64]:
#define retention in terms of dollars spent within a certain period of time after the customer’s first spend.

In [72]:
# Convert the data type of column 'DOB' from string (DD/MM/YYYY) to datetime64
df['PurchaseDate'] = pd.to_datetime(df["Date of activitiy"]).dt.strftime('%Y-%m-%d')

In [89]:
#Convert from string to date format
df['PurchaseDate'] = pd.DatetimeIndex(df['PurchaseDate']).date

In [90]:
print('Min Invoice Date:',df['PurchaseDate'].min(),'max Invoice Date:',
       df['PurchaseDate'].max())

Min Invoice Date: 2019-01-01 max Invoice Date: 2019-08-31


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174845 entries, 0 to 176426
Data columns (total 7 columns):
Supporter ID               174845 non-null int64
Email                      174845 non-null object
Date record created        174845 non-null object
Date last activity         174845 non-null object
Date of activitiy          174845 non-null object
USD currency conversion    174845 non-null float64
PurchaseDate               174845 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 10.7+ MB


In [92]:
now = dt.date(2019, 9, 1)

In [93]:
print(now)

2019-09-01


In [95]:
snapshot_date = df['PurchaseDate'].max()
snapshot_date

datetime.date(2019, 8, 31)

In [97]:
rfm = df.groupby(['Supporter ID']).agg({'PurchaseDate':lambda x:(now-x.max()).days,'Date of activitiy':'count','USD currency conversion': 'sum'})

In [98]:
rfm

,PurchaseDate,Date of activitiy,USD currency conversion
Supporter ID,,,
628110,208,1,95.15
628197,116,1,25.96
628287,74,3,338.23
628402,144,1,14.99
628415,103,1,26.11
...,...,...,...
174855413,1,1,12.16
174859505,1,1,15.00
174860564,1,1,20.00


In [99]:
#Rename columns
rfm.rename(columns={'PurchaseDate':'Recency','Date of activitiy':'Frequency','USD currency conversion':'MonetaryValue'}
           ,inplace= True)

In [100]:
rfm[(rfm['MonetaryValue']<=0)].index

Int64Index([ 13565258, 108286274, 121202705, 132435231, 144603381, 145192770,
            157869917, 160901979, 164792240, 167923206, 168196186, 172857159],
           dtype='int64', name='Supporter ID')

In [112]:
rfm[rfm['Recency']==208] #find one columns row equal to specific number

,Recency,Frequency,MonetaryValue
Supporter ID,,,
628110,208,1,95.15
2146928,208,1,228.37
3021206,208,1,60.00
3547092,208,1,34.25
3547471,208,1,35.00
...,...,...,...
159224797,208,1,100.00
159224869,208,1,60.00
159224872,208,1,100.00


In [114]:
rfm.loc[13565258] #find specific one row row 

Recency          24.0
Frequency         2.0
MonetaryValue     0.0
Name: 13565258, dtype: float64

In [115]:
#delete M = 0
rfm = rfm.drop(rfm[(rfm['MonetaryValue']<=0)].index)

In [118]:
#RFM Quartiles
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles
quantiles.to_dict()

{'Recency': {0.25: 31.0, 0.5: 82.0, 0.75: 185.0},
 'Frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 3.0},
 'MonetaryValue': {0.25: 19.575, 0.5: 35.0, 0.75: 77.25}}

In [143]:
#another way
#Building RFM segments
#Note That :

#We will rate "Recency" customer who have been active more recently better than 
#the less recent customer,because each company wants its customers to be recent

#We will rate "Frequency" and "Monetary Value" higher label because 
#we want Customer to spend more money and visit more often(that is different order than recency).
r_labels =range(4,0,-1)
f_labels=range(2,5,2)
m_labels=range(1,5)
r_quartiles = pd.qcut(rfm['Recency'], q=4, labels = r_labels)
f_quartiles = pd.qcut(rfm['Frequency'],q=4, labels = f_labels, duplicates = 'drop')
m_quartiles = pd.qcut(rfm['MonetaryValue'],q=4,labels = m_labels)
rfm = rfm.assign(R=r_quartiles,F=f_quartiles,M=m_quartiles)

# Build RFM Segment and RFM Score
def add_rfm(x) : return str(x['R']) + str(x['F']) + str(x['M'])
rfm['RFM_Segment'] = rfm.apply(add_rfm,axis=1 )
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)

rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
Supporter ID,,,,,,,,
628110,208,1,95.15,1,2,4,124,7.0
628197,116,1,25.96,2,2,2,222,6.0
628287,74,3,338.23,3,2,4,324,9.0
628402,144,1,14.99,2,2,1,221,5.0
628415,103,1,26.11,2,2,2,222,6.0


In [144]:
rfm.groupby(['F']).size().sort_values(ascending=False)[:20]

F
2    49106
4    14793
dtype: int64

In [145]:
#Use RFM score to group customers into Gold, Silver and Bronze segments:
def segments(df):
    if df['RFM_Score'] > 9 :
        return 'Gold'
    elif (df['RFM_Score'] > 5) and (df['RFM_Score'] <= 9 ):
        return 'Sliver'
    else:  
        return 'Bronze'

rfm['General_Segment'] = rfm.apply(segments,axis=1)

rfm.groupby('General_Segment').agg({'Recency':'mean','Frequency':'mean',
                                    'MonetaryValue':['mean','count']}).round(1)

Recency Frequency MonetaryValue       
                   mean      mean          mean  count
General_Segment                                       
Bronze            191.8       1.1          17.8  16473
Gold               23.3       7.5         159.7  14567
Sliver             98.0       1.5          54.1  32859

In [146]:
rfm

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score,General_Segment
Supporter ID,,,,,,,,,
628110,208,1,95.15,1,2,4,124,7.0,Sliver
628197,116,1,25.96,2,2,2,222,6.0,Sliver
628287,74,3,338.23,3,2,4,324,9.0,Sliver
628402,144,1,14.99,2,2,1,221,5.0,Bronze
628415,103,1,26.11,2,2,2,222,6.0,Sliver
...,...,...,...,...,...,...,...,...,...
174855413,1,1,12.16,4,2,1,421,7.0,Sliver
174859505,1,1,15.00,4,2,1,421,7.0,Sliver
174860564,1,1,20.00,4,2,2,422,8.0,Sliver


In [170]:
df=df.merge(rfm['General_Segment'], how='inner', on='Supporter ID')

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174822 entries, 0 to 174821
Data columns (total 8 columns):
Supporter ID               174822 non-null int64
Email                      174822 non-null object
Date record created        174822 non-null object
Date last activity         174822 non-null object
Date of activitiy          174822 non-null object
USD currency conversion    174822 non-null float64
PurchaseDate               174822 non-null object
General_Segment            174822 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 12.0+ MB


In [172]:
#get the email list segement
email_list = df.drop_duplicates('Email')

In [173]:
email_list

,Supporter ID,Email,Date record created,Date last activity,Date of activitiy,USD currency conversion,PurchaseDate,General_Segment
0,628110,hwaddell@shaw.ca,11/7/2008,9/15/2019,2/5/2019,95.15,2019-02-05,Sliver
1,628197,zarnett4therapy@sympatico.ca,11/7/2008,9/4/2019,5/8/2019,25.96,2019-05-08,Sliver
2,628287,bosatin@aol.com,11/7/2008,6/22/2019,6/19/2019,94.31,2019-06-19,Sliver
5,628402,catandfiddle@sasktel.net,11/7/2008,7/26/2019,4/10/2019,14.99,2019-04-10,Bronze
6,628415,cycarax@gmail.com,11/7/2008,9/15/2019,5/21/2019,26.11,2019-05-21,Sliver
...,...,...,...,...,...,...,...,...
174817,174855413,tamsinspargo@aol.com,8/31/2019,9/3/2019,8/31/2019,12.16,2019-08-31,Sliver
174818,174859505,jim.atkins@outlook.com,8/31/2019,9/3/2019,8/31/2019,15.00,2019-08-31,Sliver
174819,174860564,diandpeteward@icloud.com,8/31/2019,9/3/2019,8/31/2019,20.00,2019-08-31,Sliver
174820,174865864,lea@emersonmusicarts.com,8/31/2019,9/3/2019,8/31/2019,15.42,2019-08-31,Sliver
